In [ ]:
import re
import gensim
import logging
import string
import nltk
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.linear_model import Ridge

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Задание 1

In [ ]:
!wget 

## TF-IDF

Сначала уберем знаки препинания (они нам не нужны для определения категории текста), а также приведем все буквы к строчным. Соберем все в токены.

In [ ]:
from nltk.tokenize import word_tokenize

def process_text(text):
    return [word for word in word_tokenize(text.lower()) if word not in string.punctuation]

In [ ]:
X = pd.read_csv('/content/drive/MyDrive/NLP/train_data.csv')
X.head()

,id,message
0,271828,Over $616 million in Bitcoin was electrocated ...
1,271829,Quiz: Thursday or friday?
2,271830,The Australian Revenue Authority will start co...
3,271831,Let's continue😉. I present to you my new review
4,271832,Here comes your future palette.


In [ ]:
y = pd.read_csv('/content/drive/MyDrive/NLP/train_solution.csv')
y_train = y['category']
y_train = np.array(y_train)
y_train

array([1, 0, 1, ..., 2, 0, 0])

In [ ]:
X['message'] = X['message'].apply(process_text)
X['message']

0       [over, 616, million, in, bitcoin, was, electro...
1                            [quiz, thursday, or, friday]
2       [the, australian, revenue, authority, will, st...
3       [let, 's, continue😉, i, present, to, you, my, ...
4                    [here, comes, your, future, palette]
                              ...                        
3839    [on, the, air, the, shooter, 's, kms, and, the...
3840                [this, shit, that, 's, off-the-books]
3841                  [time, to, go, to, the, sky, tabor]
3842                                         [uraaaaaaaa]
3843                      [albert, i, only, drink, vodka]
Name: message, Length: 3844, dtype: object

## Сделаем TF-IDF + обучим логрег

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X['message'].apply(lambda x: " ".join(x)))

In [ ]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [ ]:
X_test = pd.read_csv('/content/drive/MyDrive/NLP/test_data.csv')
X_test['message'] = X_test['message'].apply(process_text)
X_test['message']

0       [but, a, lot, of, people, have, a, job, fair, ...
1       [also, i, got, only, 4, answers, on, the, goog...
2               [vladimir, when, will, we, have, seminar]
3                               [a, couple, at, 111, too]
4             [it, 's, on, anti-buying, and, again, zara]
                              ...                        
5922    [the, second, thing, write, all, questions, an...
5923    [can, grayscale, actually, sell, bitcoin, btc,...
5924    [it, 's, not, a, very, operational, dude, putt...
5925    [inspired, by, sonia, miro, 's, goddess, and, ...
5926        [okay, you, do, n't, have, to, worry, i, did]
Name: message, Length: 5927, dtype: object

In [ ]:
X_test = vectorizer.transform(X_test['message'].apply(lambda x: " ".join(x)))
vectorizer.get_feature_names_out()

array(['00', '000', '00000', ..., 'zuckerberg', 'über', 'überhaupt'],
      dtype=object)

In [ ]:
pred = model.predict(X_test)
pred

array([0, 0, 0, ..., 0, 2, 0])

In [ ]:
from collections import Counter

Counter(pred)

Counter({0: 3725, 2: 1280, 1: 922})

In [ ]:
ans = pd.DataFrame(columns=['id', 'category'])

In [ ]:
X =  pd.read_csv('/content/drive/MyDrive/NLP/test_data.csv')
ans['id'] = X['id']

In [ ]:
ans['category'] = pred
ans

,id,category
0,275672,0
1,275673,0
2,275674,0
3,275675,0
4,275676,0
...,...,...
5922,3143750,0
5923,3143751,1
5924,3143752,0
5925,3143753,2


In [ ]:
ans.to_csv('submit.csv', index=False)

## Теперь попробуем Word2Vec

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/NLP/train_data.csv')
X_train['message'] = X_train['message'].apply(process_text)

X_test = pd.read_csv('/content/drive/MyDrive/NLP/test_data.csv')
X_test['message'] = X_test['message'].apply(process_text)

In [ ]:
from gensim.models import Word2Vec

embedding = 300

words = Word2Vec(sentences=np.concatenate((X_train['message'], X_test['message']), axis=0), size=embedding)

In [ ]:
def w2v(txt):
    txt = [i for i in txt if i in words.wv]
    if len(txt) == 0:
        return [0]*embedding
    return np.mean(words.wv[txt], axis=0)

In [ ]:
X_train = X_train['message'].apply(w2v)
X_test = X_test['message'].apply(w2v)

In [ ]:
X_train = pd.DataFrame.from_dict(X_train.to_dict()).T
X_test = pd.DataFrame.from_dict(X_test.to_dict()).T

In [ ]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

pred = model.predict(X_test)
pred

array([2, 0, 0, ..., 0, 2, 0])

In [ ]:
Counter(pred)

Counter({2: 1778, 0: 2851, 1: 1298})

In [ ]:
ans = pd.DataFrame(columns=['id', 'category'])

In [ ]:
X =  pd.read_csv('/content/drive/MyDrive/NLP/test_data.csv')
ans['id'] = X['id']

In [ ]:
ans['category'] = pred
ans

,id,category
0,275672,2
1,275673,0
2,275674,0
3,275675,2
4,275676,2
...,...,...
5922,3143750,2
5923,3143751,1
5924,3143752,0
5925,3143753,2


In [ ]:
ans.to_csv('submit2.csv', index=False)

# Задание 2

In [ ]:
phrase = pd.Series(['my', 'future'])
sim = words.wv.most_similar(phrase, topn=1)
sim

[('trash', 0.957145631313324)]

In [ ]:
w2v('My future')

array([ 4.93754540e-03, -2.00273208e-02, -7.51304731e-04, -9.58636254e-02,
       -1.49277285e-01,  9.59441066e-03, -2.18918617e-03,  6.32570162e-02,
        7.46581703e-02,  2.17495915e-02,  3.41256969e-02,  3.50900851e-02,
       -2.75410824e-02, -4.89791892e-02, -2.89546121e-02, -3.81655917e-02,
       -1.64083932e-02, -4.24307548e-02, -9.11775753e-02,  2.36013699e-02,
        3.06598861e-02,  1.15309209e-02,  5.16322218e-02,  1.67705212e-02,
        5.87272421e-02, -3.23107652e-02,  5.45779383e-03,  4.57057692e-02,
        5.36367074e-02, -3.79753970e-02, -1.05741903e-01,  3.55090089e-02,
        2.93273367e-02,  1.24706179e-02, -1.02538325e-01, -7.51297921e-02,
       -7.31137246e-02,  3.03285476e-02, -6.41598925e-02,  2.28434708e-02,
       -7.41198808e-02,  1.23998001e-02, -1.38881877e-02,  1.35325026e-02,
        7.09258541e-02, -5.58943413e-02, -3.24541405e-02,  6.17474653e-02,
        3.76192592e-02, -9.08281375e-03,  3.28550935e-02, -2.32191626e-02,
       -2.08361410e-02,  